In [45]:
import sys
sys.path.append('/Users/matthijstentije/OpenDutchWordnet')

from __init__ import Wn_grid_parser

In [46]:
self = Wn_grid_parser(Wn_grid_parser.odwn)

In [47]:
instance = Wn_grid_parser(Wn_grid_parser.odwn)

In [48]:
adjectives = [le.get_lemma() for le in instance.les_get_generator() if le.get_pos() == 'adjective']

In [50]:
instance.lemma_get_generator("man")

In [52]:
check = ['zorgzaam', 'sterk', 'ijdel', 'lief', 'schattig']
check_words = [instance.lemma_get_generator(word) for word in check]
check_words

[[],
 [],
 [],
 [<le.Le at 0x3410ca9f0>, <le.Le at 0x3410caea0>, <le.Le at 0x341098950>],
 []]

In [119]:
alle_man_varianten = [le for le in instance.les_get_generator() if le.get_lemma() == "man"]

# Extract all the entries with "man" and "woman "
man_entries = [(le) for le in instance.les_get_generator() if le.get_lemma() == "man"]
vrouw_entries = [le for le in instance.les_get_generator() if le.get_lemma() == "vrouw"]

# Get an imporession of word, Point of Speech and synset id
for le in man_entries:
    print(f"Woord: {le.get_lemma()} - Woordsoort: {le.get_pos()} - Synset ID: {le.get_synset_id()}")

# extract de synset-ID's
man_synset_ids = set(le.get_synset_id() for le in man_entries if le.get_synset_id() is not None) # set extract unique entries
vrouw_synset_ids = set(le.get_synset_id() for le in vrouw_entries if le.get_synset_id() is not None)

    
def get_synset_info(instance, synset_id):
    synset = instance.synsets_find_synset(synset_id)
    print(synset)
    info = {}
    # Extract definition
    try:
        info['definitie'] = synset.get_glosses()
    except AttributeError:
        info['definitie'] = "Definitie niet beschikbaar"
    
    # Look if there are hypernonyms
    try:
        hyper_rel = synset.get_relations("has_hyperonym")
        info['hyperoniemen'] = [rel.get_target() for rel in hyper_rel]
    except AttributeError:
        info['hyperoniemen'] = []
    try:
        hyper_rel = synset.get_relations("has_hyponym")
        info['hyponym'] = [rel.get_target() for rel in hyper_rel]
    except AttributeError:
        info['hyponym'] = [] 
    
    return info

man_synset_info = {sid: get_synset_info(instance, sid) for sid in man_synset_ids}
vrouw_synset_info = {sid: get_synset_info(instance, sid) for sid in vrouw_synset_ids}


Woord: man - Woordsoort: noun - Synset ID: eng-30-10193967-n
Woord: man - Woordsoort: noun - Synset ID: eng-30-01321230-n
Woord: man - Woordsoort: noun - Synset ID: eng-30-00007846-n
Woord: man - Woordsoort: noun - Synset ID: eng-30-10287213-n
Woord: man - Woordsoort: noun - Synset ID: odwn-10-109967524-n


In [110]:
def extract_single_dutch_words(synset_info):
    """ Extracts only single Dutch words from the first definition of a given synset dictionary. """
    words = synset_info.get('eng-30-10287213-n', {}).get('definitie', [])
    
    # Remove duplicates
    words = list(set(words))  
    
    # Keep only single words (no phrases)
    single_words = [word for word in words if " " not in word]  
    
    return single_words

# Run the function
single_words = extract_single_dutch_words(man_synset_info)

# Print the result
print(single_words)


['kerel', 'mens', 'guy', 'vent', 'man']


In [116]:
# Stap 1: Maak een instantie van de parser
instance = Wn_grid_parser(Wn_grid_parser.odwn)

# Stap 2: Verzamel alle lexicale eenheden (lemmas) met het lemma "man"
man_entries = [le for le in instance.les_get_generator() if le.get_lemma() == "man"]

# Stap 3: Groepeer de gevonden entries per synset-ID
from collections import defaultdict
synset_entries = defaultdict(list)
for le in man_entries:
    synset_entries[le.get_synset_id()].append(le)

# (Handmatige inspectie of extra statistieken kunnen helpen bepalen welke synset de 'centroid' is)
# Stel dat we, na inspectie, concluderen dat synset_id 'eng-30-00007846-n' de centrale betekenis bevat.
centroid_synset_id = 'eng-30-10193967-n'
centroid_synset = instance.synsets_find_synset(centroid_synset_id)

# Stap 4: Haal alle woorden (lexicale eenheden) op uit de centroid-synset
centroid_entries = [le for le in instance.les_get_generator() if le.get_synset_id() == centroid_synset_id]
centroid_woorden = list(set(le.get_lemma() for le in centroid_entries))
print("Woorden binnen het centroid-synset voor 'man':", centroid_woorden)

# Stap 5: Breid de verzameling uit met aangrenzende termen via hyponiem-relaties
hyponiem_relations = centroid_synset.get_relations("has_hyponym")
nabijgelegen_woorden = []
for rel in hyponiem_relations:
    naburige_synset = instance.synsets_find_synset(rel.get_target())
    if naburige_synset:
        # Haal de woorden uit deze aangrenzende synset op
        ns_entries = [le for le in instance.les_get_generator() if le.get_synset_id() == naburige_synset.get_id()]
        nabijgelegen_woorden.extend([le.get_lemma() for le in ns_entries])
nabijgelegen_woorden = list(set(nabijgelegen_woorden))
print("Aangrenzende woorden (hyponiemen) rondom 'man':", nabijgelegen_woorden)


Woorden binnen het centroid-synset voor 'man': ['man', 'echtgenoot']
Aangrenzende woorden (hyponiemen) rondom 'man': ['pater familias', 'hoorndrager', 'huisvader', 'mijnheer']
